# Model (Vs0, epsilon, gamma) scan results

In [ ]:
import sys

pwd = !echo ${PWD}
sys.path.append(pwd[0]+"/../../../code/local/bin")

In [ ]:
import numpy as np
from scipy import signal
from scipy import ndimage
import math
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.ticker import FormatStrFormatter

import seppy

datapath=pwd[0]+"/../dat/"
figpath=pwd[0]+"/../fig/"

In [ ]:
# utility functions

def plotseis(d, p=0.9, valmin=1, valmax=-1, tmin=0, tmax=0.35, xmin=-600, xmax=600, grid=True, colorbar=False, curve=np.array([]), linestyle='--'):
    """Plot 2D seismic image"""
 
    vmin=np.min(d) / p
    vmax=p*np.max(d)
    
    if p<0:
        vmax=-p*np.amax(d)
        vmin=-vmax
        
    if valmin<valmax:
        vmin=valmin
        vmax=valmax
        
    nx=d.shape[0]
                
    fig=plt.figure(figsize=(6, 4),dpi=100)
    plt.imshow(np.transpose(d),interpolation='sinc',aspect="auto",extent=[xmin,xmax,tmax,tmin],cmap='Greys',vmin=vmin,vmax=vmax)
    plt.xlabel('Offsetx (m)')
    plt.ylabel('Time (sec)')
    if grid==True:
        plt.grid(color='r', linestyle='-', linewidth=0.2)
    if colorbar==True:
        plt.colorbar()
    if curve.size>0:
        plt.plot(np.linspace(xmin,xmax,nx),curve,linestyle)
    plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.3f'))
    
    plt.show()
    
def traveltime(v=1500, tshift=0.02, xmin=-600, xmax=600, nx=1201, y=300):
    """Compute travel time curve with constant velocity and fixed time shift and offsety"""
    
    tt = np.sqrt((np.linspace(xmin,xmax,nx))**2 + y**2)/v + tshift
    return tt

def lmo(d,xmin=-600,xmax=600,y=300,v=1500,tshift=0,dt=0.002):
    """Apply linear moveout to seismic data"""
    
    nx=d.shape[0]
    
    tt = traveltime(v,0,xmin,xmax,nx,y)
    lmo = np.copy(d)
        
    for ix in range(nx):
        lmo[ix,:] = ndimage.shift(d[ix,:],(-tt[ix]+tshift)/dt)
    
    return lmo
    
def envelop(d):
    """Compute the envelop via Hilbert transform along the fast axis"""
    
    return np.absolute(signal.hilbert(d,axis=-1))

In [ ]:
# load data
sep = seppy.sep()

sp=14

axes, data = sep.read_file(datapath+"ch6_data.H.bp")
das = data.reshape(axes.n,order='F').T
das = das[sp]

axes, data = sep.read_file(datapath+"ch6_data.HH")
attr = data.reshape(axes.n,order='F').T
attr = attr[sp]

axes, data = sep.read_file(datapath+"ch6_mute.H")
mute = data.reshape(axes.n,order='F').T
mute = mute[sp]

axes, data = sep.read_file(datapath+"ch6_dyGny_sp14_vs_all.H.bp")
syn = data.reshape(axes.n,order='F').T

ns = syn.shape[0]
dt = axes.d[0]

# apply mute to das and synthetic
das = das*mute
syn = syn*mute

# all the Vs0 scaling tested (see Makefile)
scalers=np.linspace(0,0.05,ns)

### Part 1: Vs0 scan using SV arrival

In [ ]:
# apply LMO to isolate the SV arrival
offset = (attr[600,15]-attr[600,12])**2 + (attr[600,16]-attr[600,13])**2 + (attr[600,17]-attr[600,14])**2
offset = math.sqrt(offset)
tshift=0.1
das_lmo = lmo(das,y=offset,v=1800,tshift=tshift, dt=dt)
syn_lmo = np.copy(syn)
for s in range(ns):
    syn_lmo[s] = lmo(syn[s],y=offset,v=1800,tshift=tshift, dt=dt)

In [ ]:
# FK denoising of DAS data

# save to seplib
sep.write_file("dtemp.H", np.transpose(das_lmo), ds=np.array([1,1]), os=np.array([0,0]), dpath=pwd[0]+"/")

# apply FK filter
!${PWD}/../../../code/local/bin/FK_FILTER.x < dtemp.H kmin=-0.008 kmax=0.008 taper=0.01 > dtemp.H.fk datapath=./

# read back
axes, das_lmo_fk = sep.read_file("dtemp.H.fk")
das_lmo_fk = das_lmo_fk.reshape(axes.n,order='F').T

# remove files from disk
!rm -f dtemp*

In [ ]:
# compute the envelop
das_lmo_env = envelop(das_lmo)
das_lmo_fk_env = envelop(das_lmo_fk)
syn_lmo_env = envelop(syn_lmo)

In [ ]:
# QC plots
plotseis(das_lmo)
plotseis(das_lmo_fk)
plotseis(syn_lmo[0])

In [ ]:
# build the cosine similarity functional
cs=np.zeros((ns))

for s in range(ns):
    d1 = envelop(das_lmo_fk[:,int((tshift+0.01)/dt):int((tshift+0.08)/dt)])
    d2 = envelop(syn_lmo[s,:,int((tshift+0.01)/dt):int((tshift+0.08)/dt)])
    cs[s] = np.dot(d1.flatten(),d2.flatten())/(np.linalg.norm(d1)*np.linalg.norm(d2))

In [ ]:
plotseis(d1,tmin=tshift+0.01,tmax=tshift+0.08)
plotseis(d2,tmin=tshift+0.01,tmax=tshift+0.08)

In [ ]:
xmax = scalers[np.argmax(cs)]
plt.plot(scalers,cs,'k')
plt.vlines(xmax,np.min(cs),np.max(cs),colors='r',linestyles='--')
plt.xlabel("Scaler")
plt.ylabel("Similarity value")
plt.ylim([np.min(cs),np.max(cs)])

print("The optimal scaler for Vs0 scan = %f" %xmax)

Using the optimal scaler above, update the Vs0 model and re-do another scan for epsilon and gamma

### Part II: $\epsilon$ and $\gamma$ scans using P- and SH-arrivals simultaneously

In [ ]:
# load data
axes, data = sep.read_file(datapath+"ch6_dyGny_sp14_aniso_all.H.bp")
syn2 = data.reshape(axes.n,order='F').T

ns2 = syn2.shape[0]

# apply mute to synthetic
syn2 = syn2*mute

# all the anisotropy scaling tested (see Makefile)
scalers2=np.linspace(1.00,0.72,ns2)

In [ ]:
# apply LMO to isolate the P- and SV-arrival
tshift=0.1
das_lmo_p = lmo(das,y=offset,v=4400,tshift=tshift, dt=dt)
das_lmo_sh = lmo(das,y=offset,v=2600,tshift=tshift, dt=dt)
syn2_lmo_p = np.copy(syn2)
syn2_lmo_sh = np.copy(syn2)
for s in range(ns2):
    syn2_lmo_p[s] = lmo(syn2[s],y=offset,v=4400,tshift=tshift, dt=dt)
    syn2_lmo_sh[s] = lmo(syn2[s],y=offset,v=2600,tshift=tshift, dt=dt)

In [ ]:
# FK denoising of DAS data
np.save('dtemp_p',das_lmo_p)
np.save('dtemp_sh',das_lmo_sh)

# save to seplib
sep.write_file("dtemp_p.H", np.transpose(das_lmo_p), ds=np.array([1,1]), os=np.array([0,0]), dpath=pwd[0]+"/")
sep.write_file("dtemp_sh.H", np.transpose(das_lmo_sh), ds=np.array([1,1]), os=np.array([0,0]), dpath=pwd[0]+"/")

# apply FK filter
!${PWD}/../../../code/local/bin/FK_FILTER.x < dtemp_p.H kmin=-0.008 kmax=0.008 taper=0.01 > dtemp_p.H.fk datapath=./
!${PWD}/../../../code/local/bin/FK_FILTER.x < dtemp_sh.H kmin=-0.008 kmax=0.008 taper=0.01 > dtemp_sh.H.fk datapath=./

# read back
axes, das_lmo_p_fk = sep.read_file("dtemp_p.H.fk")
das_lmo_p_fk = das_lmo_p_fk.reshape(axes.n,order='F').T

axes, das_lmo_sh_fk = sep.read_file("dtemp_sh.H.fk")
das_lmo_sh_fk = das_lmo_sh_fk.reshape(axes.n,order='F').T

# remove files from disk
!rm -f dtemp*

In [ ]:
# compute the envelop
das_lmo_p_env = envelop(das_lmo_p)
das_lmo_sh_env = envelop(das_lmo_sh)
das_lmo_p_fk_env = envelop(das_lmo_p_fk)
das_lmo_sh_fk_env = envelop(das_lmo_sh_fk)
syn2_lmo_p_env = envelop(syn2_lmo_p)
syn2_lmo_sh_env = envelop(syn2_lmo_sh)

In [ ]:
# QC plots
plotseis(das_lmo_p)
plotseis(das_lmo_p_fk)
plotseis(syn2_lmo_p[0])

plotseis(das_lmo_sh)
plotseis(das_lmo_sh_fk)
plotseis(syn2_lmo_sh[0])

In [ ]:
# build the cosine similarity functional
cs_p=np.zeros((ns2))
cs_sh=np.zeros((ns2))

for s in range(ns2):
    d1_p = (das_lmo_p_fk[200:600,int((tshift+0.01)/dt):int((tshift+0.06)/dt)])
    d2_p = (syn2_lmo_p[s,200:600,int((tshift+0.01)/dt):int((tshift+0.06)/dt)])
    cs_p[s] = np.dot(d1_p.flatten(),d2_p.flatten())/(np.linalg.norm(d1_p)*np.linalg.norm(d2_p))

    d1_sh = (das_lmo_sh_fk[200:600,int((tshift+0.01)/dt):int((tshift+0.06)/dt)])
    d2_sh = (syn2_lmo_sh[s,200:600,int((tshift+0.01)/dt):int((tshift+0.06)/dt)])
    cs_sh[s] = np.dot(d1_sh.flatten(),d2_sh.flatten())/(np.linalg.norm(d1_sh)*np.linalg.norm(d2_sh))

In [ ]:
plotseis(d1_p,tmin=tshift+0.01,tmax=tshift+0.06, xmin=-400, xmax=0)
plotseis(d2_p,tmin=tshift+0.01,tmax=tshift+0.06, xmin=-400, xmax=0)

plotseis(d1_sh,tmin=tshift+0.01,tmax=tshift+0.06, xmin=-400, xmax=0)
plotseis(d2_sh,tmin=tshift+0.01,tmax=tshift+0.06, xmin=-400, xmax=0)

In [ ]:
xmax = scalers2[np.argmax(cs_p)]
plt.plot(scalers2,cs_p,'k')
plt.vlines(xmax,np.min(cs_p),np.max(cs_p),colors='r',linestyles='--')
plt.xlabel("Scaler")
plt.ylabel("Similarity value")
plt.ylim([np.min(cs_p),np.max(cs_p)])

print("The optimal scaler for epsilon scan = %f" %xmax)

In [ ]:
xmax = scalers2[np.argmax(cs_sh)]
plt.plot(scalers2,cs_sh,'k')
plt.vlines(xmax,np.min(cs_sh),np.max(cs_sh),colors='r',linestyles='--')
plt.xlabel("Scaler")
plt.ylabel("Similarity value")
plt.ylim([np.min(cs_sh),np.max(cs_sh)])

print("The optimal scaler for gamma scan = %f" %xmax)